<a href="https://colab.research.google.com/github/rumcrush/1st_project/blob/main/%EA%B5%AC%EB%A7%A4%EA%B0%90%EC%86%8C%20%EA%B3%A0%EA%B0%9D%20%EC%98%88%EC%B8%A1%EA%B8%B0%EB%B0%98%20%EC%86%94%EB%A3%A8%EC%85%98_%EC%A0%84%EC%B2%B4%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAW 데이터프레임 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
compuse = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/경쟁사이용.txt', encoding='CP949')
cust = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/고객DEMO.txt', encoding='CP949')
membership = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/멤버십여부.txt', encoding='CP949')
prodcat = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/상품분류.txt', encoding='CP949')
channel = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/채널이용.txt', encoding='CP949')
purprd = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/purprd_new2.csv', encoding='utf-8', index_col=0)

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# 반기별 df 만들기

In [ ]:
# 채널이용여부는 3개월 이내의 데이터라고 하나, 기준일시를 모르며,
# 알더라도 현재 모델에 적용하기에는 문제가 있음

In [ ]:
# 고객 데이터프레임에 멤버십가입년월 컬럼 추가
# 가입여부가 아니라 가입년월로 넣어놓은 이유는 반기별로 고객데이터 분할한 이후에 가입여부가 바뀔 수 있기 때문
# 예를 들면 가입년월이 201408 이면 2014년 상반기 데이터에는 멤버십가입여부를 0으로 적용하기 위함
# 여러가지 멤버십 복수가입 시, 가장 일찍 가입한 멤버십 기준으로 가입년월 집계함(min)

membership.가입년월 = pd.to_datetime(membership.가입년월, format = '%Y%m')
cust = cust.merge(membership.pivot_table(index = '고객번호', values = '가입년월', aggfunc='min'), how = 'left', on = '고객번호').rename({'가입년월':'멤버십가입년월'}, axis = 1)
# cust.멤버십가입년월 = cust.멤버십가입년월.fillna(0)
cust.head(3)

,고객번호,성별,연령대,거주지역,멤버십가입년월
0,1,M,60세이상,60.0,NaT
1,2,M,60세이상,100.0,NaT
2,3,M,60세이상,33.0,NaT


In [ ]:
# 경쟁사 이용년월 min 값은 201501이므로 학습 데이터에 사용할 수 없다.

In [ ]:
# 고객 데이터프레임 반기별로 분할
cust_14_half1 = cust.copy()
cust_14_half2 = cust.copy()
cust_15_half1 = cust.copy()
cust_15_half2 = cust.copy()

In [ ]:
# purprd 데이터프레임 수정

# 구매일자 datetime 형식으로 변환
purprd.구매일자 = pd.to_datetime(purprd.구매일자, format = '%Y%m%d')

# 구매시간 범주화
def time_cat(x):
    if 9 <= x < 12:
        return '오전'
    elif 12 <= x < 15:
        return '낮'
    elif 15 <= x < 18:
        return '오후'
    elif 18 <= x < 21:
        return '저녁'
    else:
        return '야간'
purprd.구매시간 = purprd.구매시간.apply(time_cat)

# 점포코드를 제휴사와 합친 형태로 변환 (ex. 제휴사 B, 점포코드 101 -> 점포코드 B101)
purprd.점포코드 = purprd.제휴사 + purprd.점포코드.astype(str)

In [ ]:
# 반기별 구매데이터프레임 분할
purprd_14_half1 = purprd[(purprd.구매일자 >= pd.to_datetime(20140101, format = '%Y%m%d')) & (purprd.구매일자 < pd.to_datetime(20140701, format = '%Y%m%d'))]
purprd_14_half2 = purprd[(purprd.구매일자 >= pd.to_datetime(20140701, format = '%Y%m%d')) & (purprd.구매일자 < pd.to_datetime(20150101, format = '%Y%m%d'))]
purprd_15_half1 = purprd[(purprd.구매일자 >= pd.to_datetime(20150101, format = '%Y%m%d')) & (purprd.구매일자 < pd.to_datetime(20150701, format = '%Y%m%d'))]
purprd_15_half2 = purprd[(purprd.구매일자 >= pd.to_datetime(20150701, format = '%Y%m%d')) & (purprd.구매일자 < pd.to_datetime(20160101, format = '%Y%m%d'))]

In [ ]:
# purprd.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/purprd_new3.csv')

In [ ]:
# 반기별 고객 데이터프레임에 각 제휴사별 구매액 비율 추가

def get_amount_rate(purprd, cust):
    temp = purprd.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0)
    temp['총계'] = temp.sum(axis = 1)
    temp = temp.apply(lambda x: x/temp['총계'])
    temp = temp.drop('총계', axis = 1)
    cust = cust.merge(temp, how = 'left', on = '고객번호').rename({'A':'A구매금액비율', 'B':'B구매금액비율', 'C':'C구매금액비율', 'D':'D구매금액비율'}, axis = 1)
    return cust

cust_14_half1 = get_amount_rate(purprd_14_half1,cust_14_half1)
cust_14_half2 = get_amount_rate(purprd_14_half2,cust_14_half2)
cust_15_half1 = get_amount_rate(purprd_15_half1,cust_15_half1)
cust_15_half2 = get_amount_rate(purprd_15_half2,cust_15_half2)

In [ ]:
# 주요방문시간대
import numpy as np
from scipy.stats import mode

def get_feq_time(purprd, cust):
    temp = purprd.groupby(['고객번호', '영수증번호']).구매시간.first().reset_index().groupby('고객번호').구매시간.apply(lambda x: mode(x)[0][0])
    cust = cust.merge(temp.rename('주요구매시간대'), how = 'left', on = '고객번호')
    return cust

cust_14_half1 = get_feq_time(purprd_14_half1,cust_14_half1)
cust_14_half2 = get_feq_time(purprd_14_half2,cust_14_half2)
cust_15_half1 = get_feq_time(purprd_15_half1,cust_15_half1)
cust_15_half2 = get_feq_time(purprd_15_half2,cust_15_half2)

In [ ]:
# 제휴사별 방문횟수
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'nunique').fillna(0).rename({'A':'A방문횟수', 'B':'B방문횟수', 'C':'C방문횟수', 'D':'D방문횟수'}, axis = 1), how = 'left', on = '고객번호')

In [ ]:
# 제휴사별 상품구매횟수
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '영수증번호', aggfunc = 'count').fillna(0).rename({'A':'A상품구매개수', 'B':'B상품구매개수', 'C':'C상품구매개수', 'D':'D상품구매개수'}, axis = 1), how = 'left', on = '고객번호')

In [ ]:
# 반기별 총구매금액
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.groupby('고객번호').구매금액.sum().rename('총구매금액'), how = 'left', on = '고객번호')

In [ ]:
# 방문점포 종류수
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.groupby('고객번호').점포코드.nunique().rename('방문점포종류수'), how = 'left', on = '고객번호')

In [ ]:
# 멤버십가입반영
def membership_check(mem_date, ref_date):
    if mem_date < ref_date:
        return 1
    else:
        return 0

ref_date = pd.to_datetime(20140701, format = '%Y%m%d')
cust_14_half1.멤버십가입년월 = cust_14_half1.멤버십가입년월.apply(lambda x: membership_check(x, ref_date))
ref_date = pd.to_datetime(20150101, format = '%Y%m%d')
cust_14_half2.멤버십가입년월 = cust_14_half2.멤버십가입년월.apply(lambda x: membership_check(x, ref_date))
ref_date = pd.to_datetime(20150701, format = '%Y%m%d')
cust_15_half1.멤버십가입년월 = cust_15_half1.멤버십가입년월.apply(lambda x: membership_check(x, ref_date))
ref_date = pd.to_datetime(20160101, format = '%Y%m%d')
cust_15_half2.멤버십가입년월 = cust_15_half2.멤버십가입년월.apply(lambda x: membership_check(x, ref_date))

cust_14_half1 = cust_14_half1.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)
cust_14_half2 = cust_14_half2.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)
cust_15_half1 = cust_15_half1.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)
cust_15_half2 = cust_15_half2.rename({'멤버십가입년월':'멤버십가입여부'}, axis = 1)

In [ ]:
# 마지막구매일자 만들기
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', values='구매일자', aggfunc = 'max').rename({'구매일자':'마지막구매일자'}, axis = 1), how = 'left', on = '고객번호')

In [ ]:
# 총구매금액 / 총 방문수 

# 총 방문수
cust_14_half1['회당구매금액'] = cust_14_half1.총구매금액/(cust_14_half1.A방문횟수 + cust_14_half1.B방문횟수 + cust_14_half1.C방문횟수 + cust_14_half1.D방문횟수)
cust_14_half2['회당구매금액'] = cust_14_half2.총구매금액/(cust_14_half2.A방문횟수 + cust_14_half2.B방문횟수 + cust_14_half2.C방문횟수 + cust_14_half2.D방문횟수)
cust_15_half1['회당구매금액'] = cust_15_half1.총구매금액/(cust_15_half1.A방문횟수 + cust_15_half1.B방문횟수 + cust_15_half1.C방문횟수 + cust_15_half1.D방문횟수)
cust_15_half2['회당구매금액'] = cust_15_half2.총구매금액/(cust_15_half2.A방문횟수 + cust_15_half2.B방문횟수 + cust_15_half2.C방문횟수 + cust_15_half2.D방문횟수)

In [ ]:
# 구매상품종류(소분류기준) 정리
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', values = '소분류코드', aggfunc = 'nunique').rename({'소분류코드':'구매상품종류수'}, axis = 1), how = 'left', on = '고객번호')

In [ ]:
# 제휴사별 구매금액 추가
cust_14_half1 = cust_14_half1.merge(purprd_14_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')
cust_14_half2 = cust_14_half2.merge(purprd_14_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half1 = cust_15_half1.merge(purprd_15_half1.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')
cust_15_half2 = cust_15_half2.merge(purprd_15_half2.pivot_table(index = '고객번호', columns = '제휴사', values = '구매금액', aggfunc = 'sum').fillna(0).rename({'A':'A구매금액', 'B':'B구매금액', 'C':'C구매금액', 'D':'D구매금액'}, axis = 1), how = 'left', on = '고객번호')

In [ ]:
# 분기별 랭크추가
# 반기별 구매데이터프레임 분할
bd1 = pd.to_datetime('20140401', format = '%Y%m%d')
bd2 = pd.to_datetime('20140701', format = '%Y%m%d')
bd3 = pd.to_datetime('20141001', format = '%Y%m%d')
bd4 = pd.to_datetime('20150101', format = '%Y%m%d')
bd5 = pd.to_datetime('20150401', format = '%Y%m%d')
bd6 = pd.to_datetime('20150701', format = '%Y%m%d')
bd7 = pd.to_datetime('20151001', format = '%Y%m%d')
bd8 = pd.to_datetime('20160101', format = '%Y%m%d')

fh14_1 = purprd[purprd.구매일자<bd1]
fh14_2 = purprd[(purprd.구매일자>=bd1)&(purprd.구매일자<bd2)]
sh14_1 = purprd[(purprd.구매일자>=bd2)&(purprd.구매일자<bd3)]
sh14_2 = purprd[(purprd.구매일자>=bd3)&(purprd.구매일자<bd4)]

fh15_1 = purprd[(purprd.구매일자>=bd4)&(purprd.구매일자<bd5)]
fh15_2 = purprd[(purprd.구매일자>=bd5)&(purprd.구매일자<bd6)]
sh15_1 = purprd[(purprd.구매일자>=bd6)&(purprd.구매일자<bd7)]
sh15_2 = purprd[(purprd.구매일자>=bd7)&(purprd.구매일자<bd8)]

# 순위적용
fh14_1_pv = fh14_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
fh14_1_pv = fh14_1_pv.rank(method='min', ascending=True).fillna(1)

fh14_2_pv = fh14_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
fh14_2_pv = fh14_2_pv.rank(method='min', ascending=True).fillna(1)

sh14_1_pv = sh14_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
sh14_1_pv = sh14_1_pv.rank(method='min', ascending=True).fillna(1)

sh14_2_pv = sh14_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
sh14_2_pv = sh14_2_pv.rank(method='min', ascending=True).fillna(1)

fh15_1_pv = fh15_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
fh15_1_pv = fh15_1_pv.rank(method='min', ascending=True).fillna(1)

fh15_2_pv = fh15_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
fh15_2_pv = fh15_2_pv.rank(method='min', ascending=True).fillna(1)

sh15_1_pv = sh15_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
sh15_1_pv = sh15_1_pv.rank(method='min', ascending=True).fillna(1)

sh15_2_pv = sh15_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
sh15_2_pv = sh15_2_pv.rank(method='min', ascending=True).fillna(1)

# 등급적용
def rank(x):
    if x <= 4846:
        return 1
    elif x <= 9692:
        return 2
    elif x <= 14539:
        return 3
    else:
        return 4

n_list = [fh14_1_pv,fh14_2_pv,sh14_1_pv,sh14_2_pv,fh15_1_pv,fh15_2_pv, sh15_1_pv,sh15_2_pv]
for el in n_list:
    for i in range(1,13):
        el[i] = el[i].apply(lambda x:rank(x))

# 반기별 데이터프레임에 합병
cust_14_half1 = cust_14_half1.merge(fh14_1_pv, on='고객번호')
cust_14_half1 = cust_14_half1.merge(fh14_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_14_half2 = cust_14_half2.merge(sh14_1_pv, on='고객번호')
cust_14_half2 = cust_14_half2.merge(sh14_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_15_half1 = cust_15_half1.merge(fh15_1_pv, on='고객번호')
cust_15_half1 = cust_15_half1.merge(fh15_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_15_half2 = cust_15_half2.merge(sh15_1_pv, on='고객번호')
cust_15_half2 = cust_15_half2.merge(sh15_2_pv, on='고객번호', suffixes=['_1','_2'])

In [ ]:
# cust_14_half1.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_14_half1_3.csv')
# cust_14_half2.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_14_half2_3.csv')
# cust_15_half1.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_15_half1_3.csv')
# cust_15_half2.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_15_half2_3.csv')

In [ ]:
# 2014-1반기 기준, 고객별 총매출액 보정

#14년 1반기 -15년도 1반기 총매출액
s1 = cust_14_half1.총구매금액.sum()
s2 = cust_14_half2.총구매금액.sum()
s3 = cust_15_half1.총구매금액.sum()
s4 = cust_15_half2.총구매금액.sum()

# 2014-1반기 기준, 각 반기의 매출 증감에 대한 보정 비율
r2 = s1 / s2
r3 = s1 / s3
r4 = s1 / s4
print(r2)
print(r3)
print(r4)
print()
# display(cust_14_half2.총구매금액.head(3), cust_15_half1.총구매금액.head(3), cust_15_half2.총구매금액.head(3))

# 2014-1반기 대비 매출 증감 보정
cust_14_half2.총구매금액 = cust_14_half2.총구매금액*r2
cust_15_half1.총구매금액 = cust_15_half1.총구매금액*r3
cust_15_half2.총구매금액 = cust_15_half2.총구매금액*r4

0.922970402735531
0.933810401208944
0.887298427047888



In [ ]:
# 1회 방문당 구매금액
#총구매금액 / 총 방문수 

cust_14_half1['회당구매금액'] = cust_14_half1.총구매금액/(cust_14_half1.A방문횟수 + cust_14_half1.B방문횟수 + cust_14_half1.C방문횟수 + cust_14_half1.D방문횟수)
cust_14_half2['회당구매금액'] = cust_14_half2.총구매금액/(cust_14_half2.A방문횟수 + cust_14_half2.B방문횟수 + cust_14_half2.C방문횟수 + cust_14_half2.D방문횟수)
cust_15_half1['회당구매금액'] = cust_15_half1.총구매금액/(cust_15_half1.A방문횟수 + cust_15_half1.B방문횟수 + cust_15_half1.C방문횟수 + cust_15_half1.D방문횟수)
cust_15_half2['회당구매금액'] = cust_15_half2.총구매금액/(cust_15_half2.A방문횟수 + cust_15_half2.B방문횟수 + cust_15_half2.C방문횟수 + cust_15_half2.D방문횟수)

## csv 파일 업데이트: _1

In [ ]:
# #변경 내용csv 저장(cust_14_half1_1)
# cust_14_half1.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_14_half1_1.csv')
# cust_14_half2.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_14_half2_1.csv')
# cust_15_half1.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_15_half1_1.csv')
# cust_15_half2.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_15_half2_1.csv')

In [ ]:
# 분기, 대분류별 구매순위 등급(4등급)
# 반기별 구매데이터프레임 분할
# fh: first half, sh: second half

purprd2 = purprd
fh14_1 = purprd2[purprd2.구매일자<'20140401']
fh14_2 = purprd2[(purprd2.구매일자>='20140401')&(purprd2.구매일자<'20140701')]
sh14_1 = purprd2[(purprd2.구매일자>='20140701')&(purprd2.구매일자<'20141001')]
sh14_2 = purprd2[(purprd2.구매일자>='20141001')&(purprd2.구매일자<'20150101')]

fh15_1 = purprd2[(purprd2.구매일자>='20150101')&(purprd2.구매일자<'20150401')]
fh15_2 = purprd2[(purprd2.구매일자>='20150401')&(purprd2.구매일자<'20150701')]
sh15_1 = purprd2[(purprd2.구매일자>='20150701')&(purprd2.구매일자<'20151001')]
sh15_2 = purprd2[(purprd2.구매일자>='20151001')&(purprd2.구매일자<'20160101')]

In [ ]:
# 고객별 대분류별 구매금액 총액에 대한 rank 매기기(오름차순)
# 카테고리 구매가 없는 고객은 모두 1위로 지정

fh14_1_pv = fh14_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
fh14_1_pv = fh14_1_pv.rank(method='min', ascending=True).fillna(1)

fh14_2_pv = fh14_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
fh14_2_pv = fh14_2_pv.rank(method='min', ascending=True).fillna(1)

sh14_1_pv = sh14_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
sh14_1_pv = sh14_1_pv.rank(method='min', ascending=True).fillna(1)

sh14_2_pv = sh14_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
sh14_2_pv = sh14_2_pv.rank(method='min', ascending=True).fillna(1)

fh15_1_pv = fh15_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
fh15_1_pv = fh15_1_pv.rank(method='min', ascending=True).fillna(1)

fh15_2_pv = fh15_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
fh15_2_pv = fh15_2_pv.rank(method='min', ascending=True).fillna(1)

sh15_1_pv = sh15_1.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
sh15_1_pv = sh15_1_pv.rank(method='min', ascending=True).fillna(1)

sh15_2_pv = sh15_2.groupby(['고객번호','대대분류코드'])['구매금액'].sum().unstack()
sh15_2_pv = sh15_2_pv.rank(method='min', ascending=True).fillna(1)

In [ ]:
# 등수에 따라 4등급으로 분류하는 함수 작성
def rank(x):
  if x <= 4846:
    return 1
  elif x <= 9692:
    return 2
  elif x <= 14539:
    return 3
  else:
    return 4

# 전체 8분기, 각 대분류에 대한 등급 매기기
n_list = [fh14_1_pv,fh14_2_pv,sh14_1_pv,sh14_2_pv,fh15_1_pv,fh15_2_pv, sh15_1_pv,sh15_2_pv]

for el in n_list:
  for i in range(1,13):
    el[i] = el[i].apply(lambda x:rank(x))

In [ ]:
# # 등수에 따라 10등급으로 분류하는 함수 작성
# def rank(x):
#   if x <= 1938:
#     return 1
#   elif x <= 3876:
#     return 2
#   elif x <= 5814:
#     return 3
#   elif x <= 7752:
#     return 4
#   elif x <= 9690:
#     return 5
#   elif x <= 11628:
#     return 6
#   elif x <= 13566:
#     return 7
#   elif x <= 15504:
#     return 8
#   elif x <= 17742:
#     return 9
#   else:
#     return 10

# # 전체 8분기, 각 대분류에 대한 등급 매기기
# n_list = [fh14_1_pv,fh14_2_pv,sh14_1_pv,sh14_2_pv,fh15_1_pv,fh15_2_pv, sh15_1_pv,sh15_2_pv]

# for el in n_list:
#   for i in range(1,13):
#     el[i] = el[i].apply(lambda x:rank(x))

In [ ]:
# csv 파일(버전:_1) 불러오기
cust_14_half1_1 = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_14_half1_1.csv', encoding='utf-8', index_col=0)
cust_14_half2_1 = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_14_half2_1.csv', encoding='utf-8', index_col=0)
cust_15_half1_1 = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_15_half1_1.csv', encoding='utf-8', index_col=0)
cust_15_half2_1 = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_15_half2_1.csv', encoding='utf-8', index_col=0)

In [ ]:
# 버전_1 df에 대분류별 구매순위 등급 넣기
cust_14_half1_2 = cust_14_half1_1.merge(fh14_1_pv, on='고객번호')
cust_14_half1_2 = cust_14_half1_2.merge(fh14_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_14_half2_2 = cust_14_half2_1.merge(sh14_1_pv, on='고객번호')
cust_14_half2_2 = cust_14_half2_2.merge(sh14_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_15_half1_2 = cust_15_half1_1.merge(fh15_1_pv, on='고객번호')
cust_15_half1_2 = cust_15_half1_2.merge(fh15_2_pv, on='고객번호', suffixes=['_1','_2'])

cust_15_half2_2 = cust_15_half2_1.merge(sh15_1_pv, on='고객번호')
cust_15_half2_2 = cust_15_half2_2.merge(sh15_2_pv, on='고객번호', suffixes=['_1','_2'])

## csv 파일 업데이트: _2

In [ ]:
# cust_14_half1_2.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_14_half1_2.csv')
# cust_14_half2_2.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_14_half2_2.csv')
# cust_15_half1_2.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_15_half1_2.csv')
# cust_15_half2_2.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/cust_15_half2_2.csv')

# 데이터 전처리

In [ ]:
# 인덱스를 고객번호로 지정
for i in [cust_14_half1_2, cust_14_half2_2, cust_15_half1_2, cust_15_half2_2]:
    i.set_index('고객번호', inplace = True)

In [ ]:
# 거주지역 drop
cust_14_half1_2.drop('거주지역', axis = 1, inplace = True)
cust_14_half2_2.drop('거주지역', axis = 1, inplace = True)
cust_15_half1_2.drop('거주지역', axis = 1, inplace = True)
cust_15_half2_2.drop('거주지역', axis = 1, inplace = True)

In [ ]:
cust_14_half1_2.마지막구매일자 = cust_14_half1_2.마지막구매일자.astype('datetime64')
cust_14_half2_2.마지막구매일자 = cust_14_half2_2.마지막구매일자.astype('datetime64')
cust_15_half1_2.마지막구매일자 = cust_15_half1_2.마지막구매일자.astype('datetime64')
cust_15_half2_2.마지막구매일자 = cust_15_half2_2.마지막구매일자.astype('datetime64')

In [ ]:
# Label Encoding
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cust_14_half1_2.성별 = le.fit_transform(cust_14_half1_2.성별)
cust_14_half2_2.성별 = le.transform(cust_14_half2_2.성별)
cust_15_half1_2.성별 = le.transform(cust_15_half1_2.성별)
cust_15_half2_2.성별 = le.transform(cust_15_half2_2.성별)

In [ ]:
# One-hot Encoding
cust_14_half1_2 = pd.get_dummies(cust_14_half1_2)
cust_14_half2_2 = pd.get_dummies(cust_14_half2_2)
cust_15_half1_2 = pd.get_dummies(cust_15_half1_2)
cust_15_half2_2 = pd.get_dummies(cust_15_half2_2)

# y_train, y_test 만들기

In [ ]:
# 종속변수 만들기
y_train = cust_15_half1_2.총구매금액 - cust_14_half2_2.총구매금액
y_test = cust_15_half2_2.총구매금액 - cust_15_half1_2.총구매금액

# 이름바꾸기
y_train.rename('구매증감', inplace = True)
y_test.rename('구매증감', inplace = True)

def label_decision(x):
    if x < 0:
        return 1
    else:
        return 0

y_train = y_train.apply(label_decision)
y_test = y_test.apply(label_decision)

# x_train 만들기

In [ ]:
# 훈련 데이터 만들기
x_train = cust_14_half2_2.copy()

x_train = x_train.drop(['1_1', '2_1', '3_1', '4_1',
       '5_1', '6_1', '7_1', '8_1', '9_1', '10_1', '11_1', '12_1', '1_2', '2_2',
       '3_2', '4_2', '5_2', '6_2', '7_2', '8_2', '9_2', '10_2', '11_2', '12_2'], axis = 1)
x_train.iloc[:, -5:] = cust_14_half2_2.iloc[:, -5:] + cust_14_half1_2.iloc[:, -5:]
x_train[['A구매금액비율평균', 'B구매금액비율평균', 'C구매금액비율평균', 'D구매금액비율평균']] = \
(cust_14_half2_2[['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율']] + cust_14_half1_2[['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율']])/2
x_train[['A구매금액비율변동', 'B구매금액비율변동', 'C구매금액비율변동', 'D구매금액비율변동']] = \
cust_14_half2_2[['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율']] - cust_14_half1_2[['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율']]
x_train[['총구매금액평균']] = \
(cust_14_half2_2[['총구매금액']] + cust_14_half1_2[['총구매금액']])/2
x_train[['총구매금액변동']] = \
cust_14_half2_2[['총구매금액']] - cust_14_half1_2[['총구매금액']]
x_train[['A방문횟수평균', 'B방문횟수평균', 'C방문횟수평균', 'D방문횟수평균']] = \
(cust_14_half2_2[['A방문횟수', 'B방문횟수', 'C방문횟수', 'D방문횟수']] + cust_14_half1_2[['A방문횟수', 'B방문횟수', 'C방문횟수', 'D방문횟수']])/2
x_train[['A방문횟수변동', 'B방문횟수변동', 'C방문횟수변동', 'D방문횟수변동']] = \
cust_14_half2_2[['A방문횟수', 'B방문횟수', 'C방문횟수', 'D방문횟수']] - cust_14_half1_2[['A방문횟수', 'B방문횟수', 'C방문횟수', 'D방문횟수']]
x_train[['A상품구매개수평균', 'B상품구매개수평균', 'C상품구매개수평균', 'D상품구매개수평균']] = \
(cust_14_half2_2[['A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수']] + cust_14_half1_2[['A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수']])/2
x_train[['A상품구매개수변동', 'B상품구매개수변동', 'C상품구매개수변동', 'D상품구매개수변동']] = \
cust_14_half2_2[['A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수']] - cust_14_half1_2[['A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수']]
x_train[['방문점포종류수평균']] = \
(cust_14_half2_2[['방문점포종류수']] + cust_14_half1_2[['방문점포종류수']])/2
x_train[['방문점포종류수변동']] = \
cust_14_half2_2[['방문점포종류수']] - cust_14_half1_2[['방문점포종류수']]
x_train[['회당구매금액평균']] = \
(cust_14_half2_2[['회당구매금액']] + cust_14_half1_2[['회당구매금액']])/2
x_train[['회당구매금액변동']] = \
cust_14_half2_2[['회당구매금액']] - cust_14_half1_2[['회당구매금액']]
x_train['recency'] = \
(pd.to_datetime('20150101', format = '%Y%m%d') - cust_14_half2_2['마지막구매일자']).apply(lambda x: x.days)
x_train = x_train.drop(['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율', 'A방문횟수',
       'B방문횟수', 'C방문횟수', 'D방문횟수', 'A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수',
       '총구매금액', '방문점포종류수', '회당구매금액', '마지막구매일자'], axis = 1)
x_train

,성별,멤버십가입여부,구매상품종류수,연령대_19세이하,연령대_20세~24세,연령대_25세~29세,연령대_30세~34세,연령대_35세~39세,연령대_40세~44세,연령대_45세~49세,연령대_50세~54세,연령대_55세~59세,연령대_60세이상,주요구매시간대_낮,주요구매시간대_야간,주요구매시간대_오전,주요구매시간대_오후,주요구매시간대_저녁,A구매금액비율평균,B구매금액비율평균,C구매금액비율평균,D구매금액비율평균,A구매금액비율변동,B구매금액비율변동,C구매금액비율변동,D구매금액비율변동,총구매금액평균,총구매금액변동,A방문횟수평균,B방문횟수평균,C방문횟수평균,D방문횟수평균,A방문횟수변동,B방문횟수변동,C방문횟수변동,D방문횟수변동,A상품구매개수평균,B상품구매개수평균,C상품구매개수평균,D상품구매개수평균,A상품구매개수변동,B상품구매개수변동,C상품구매개수변동,D상품구매개수변동,방문점포종류수평균,방문점포종류수변동,회당구매금액평균,회당구매금액변동,recency
고객번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,154.0,0,0,0,0,0,0,0,0,0,1,2.0,0.0,0.0,0.0,0.0,0.983053,0.006060,0.010888,0.0,-0.011597,0.009931,0.001666,0.0,2.571476e+07,1.157882e+07,180.0,2.5,6.5,0.0,52.0,3.0,5.0,0.0,252.0,21.0,24.0,0.0,44.0,40.0,34.0,0.0,6.5,1.0,134585.685363,18537.974499,1
2,1,0,74.0,0,0,0,0,0,0,0,0,0,1,1.0,0.0,0.0,1.0,0.0,0.998314,0.001686,0.000000,0.0,-0.000865,0.000865,0.000000,0.0,2.228342e+07,-4.079611e+06,176.5,1.0,0.0,0.0,-21.0,0.0,0.0,0.0,301.0,4.0,0.0,0.0,-74.0,6.0,0.0,0.0,3.0,0.0,125299.080595,-8159.604768,1
3,1,0,92.0,0,0,0,0,0,0,0,0,0,1,2.0,0.0,0.0,0.0,0.0,0.046992,0.003338,0.949670,0.0,-0.093983,0.006676,0.087307,0.0,9.069726e+05,1.115151e+05,0.5,0.5,147.0,0.0,-1.0,1.0,40.0,0.0,0.5,1.0,368.0,0.0,-1.0,2.0,42.0,0.0,2.5,1.0,6190.326873,-919.580629,2
4,0,0,106.0,0,0,0,0,0,0,0,0,0,1,2.0,0.0,0.0,0.0,0.0,0.894840,0.057351,0.047809,0.0,-0.046164,0.082945,-0.036781,0.0,4.486818e+06,-1.110575e+06,101.0,7.5,12.5,0.0,-34.0,9.0,-7.0,0.0,153.0,28.5,39.5,0.0,-96.0,27.0,-29.0,0.0,4.0,0.0,37123.422158,639.457454,1
5,1,0,30.0,0,0,0,0,0,0,0,0,0,1,2.0,0.0,0.0,0.0,0.0,0.998535,0.001465,0.000000,0.0,0.002930,-0.002930,0.000000,0.0,3.757733e+06,-3.404835e+06,147.0,0.5,0.0,0.0,-144.0,-1.0,0.0,0.0,253.0,0.5,0.0,0.0,-246.0,-1.0,0.0,0.0,1.5,-1.0,26111.534915,2585.342556,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19369,0,0,196.0,0,0,0,0,0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
19371,0,0,161.0,0,0,0,0,0,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
19372,0,0,98.0,0,0,0,0,1,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6


In [ ]:
# 분기별 카테고리 랭크 데이터 분리
q1 = cust_14_half1_2.iloc[:,19:31]
q2 = cust_14_half1_2.iloc[:,31:43]
q3 = cust_14_half2_2.iloc[:,19:31]
q4 = cust_14_half2_2.iloc[:,31:43]

In [ ]:
# 분기별 카테고리 랭크 컬럼명 통일
q1 = q1.rename({'1_1':'1', '2_1':'2', '3_1':'3', '4_1':'4',
       '5_1':'5', '6_1':'6', '7_1':'7', '8_1':'8', '9_1':'9', '10_1':'10', '11_1':'11', '12_1':'12'}, axis =1)
q2 = q2.rename({'1_2':'1', '2_2':'2', '3_2':'3', '4_2':'4',
       '5_2':'5', '6_2':'6', '7_2':'7', '8_2':'8', '9_2':'9', '10_2':'10', '11_2':'11', '12_2':'12'}, axis =1)
q3 = q3.rename({'1_1':'1', '2_1':'2', '3_1':'3', '4_1':'4',
       '5_1':'5', '6_1':'6', '7_1':'7', '8_1':'8', '9_1':'9', '10_1':'10', '11_1':'11', '12_1':'12'}, axis =1)
q4 = q4.rename({'1_2':'1', '2_2':'2', '3_2':'3', '4_2':'4',
       '5_2':'5', '6_2':'6', '7_2':'7', '8_2':'8', '9_2':'9', '10_2':'10', '11_2':'11', '12_2':'12'}, axis =1)

In [ ]:
# 카테고리 net 반영
x_train = x_train.merge((q4-q1).rename({'1':'1_net', '2':'2_net', '3':'3_net', '4':'4_net', '5':'5_net', '6':'6_net', '7':'7_net', '8':'8_net', '9':'9_net', '10':'10_net', '11':'11_net', '12':'12_net'}, axis = 1), how = 'left', on = '고객번호')

In [ ]:
# 카테고리 sum 반영
x_train = x_train.merge(((q4 - q3).apply(abs) + (q3 - q2).apply(abs) + (q2 - q1).apply(abs)).rename({'1':'1_sum', '2':'2_sum', '3':'3_sum', '4':'4_sum', '5':'5_sum', '6':'6_sum', '7':'7_sum', '8':'8_sum', '9':'9_sum', '10':'10_sum', '11':'11_sum', '12':'12_sum'}, axis = 1), how = 'left', on = '고객번호')

 # x_test 만들기

In [ ]:
# 테스트 데이터도 동일한 방식으로
x_test = cust_15_half1_2.copy()

x_test = x_test.drop(['1_1', '2_1', '3_1', '4_1',
       '5_1', '6_1', '7_1', '8_1', '9_1', '10_1', '11_1', '12_1', '1_2', '2_2',
       '3_2', '4_2', '5_2', '6_2', '7_2', '8_2', '9_2', '10_2', '11_2', '12_2'], axis = 1)
x_test.iloc[:, -5:] = cust_15_half1_2.iloc[:, -5:] + cust_14_half2_2.iloc[:, -5:]
x_test[['A구매금액비율평균', 'B구매금액비율평균', 'C구매금액비율평균', 'D구매금액비율평균']] = \
(cust_15_half1_2[['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율']] + cust_14_half2_2[['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율']])/2
x_test[['A구매금액비율변동', 'B구매금액비율변동', 'C구매금액비율변동', 'D구매금액비율변동']] = \
cust_15_half1_2[['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율']] - cust_14_half2_2[['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율']]
x_test[['총구매금액평균']] = \
(cust_15_half1_2[['총구매금액']] + cust_14_half2_2[['총구매금액']])/2
x_test[['총구매금액변동']] = \
cust_15_half1_2[['총구매금액']] - cust_14_half2_2[['총구매금액']]
x_test[['A방문횟수평균', 'B방문횟수평균', 'C방문횟수평균', 'D방문횟수평균']] = \
(cust_15_half1_2[['A방문횟수', 'B방문횟수', 'C방문횟수', 'D방문횟수']] + cust_14_half2_2[['A방문횟수', 'B방문횟수', 'C방문횟수', 'D방문횟수']])/2
x_test[['A방문횟수변동', 'B방문횟수변동', 'C방문횟수변동', 'D방문횟수변동']] = \
cust_15_half1_2[['A방문횟수', 'B방문횟수', 'C방문횟수', 'D방문횟수']] - cust_14_half2_2[['A방문횟수', 'B방문횟수', 'C방문횟수', 'D방문횟수']]
x_test[['A상품구매개수평균', 'B상품구매개수평균', 'C상품구매개수평균', 'D상품구매개수평균']] = \
(cust_15_half1_2[['A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수']] + cust_14_half2_2[['A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수']])/2
x_test[['A상품구매개수변동', 'B상품구매개수변동', 'C상품구매개수변동', 'D상품구매개수변동']] = \
cust_15_half1_2[['A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수']] - cust_14_half2_2[['A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수']]
x_test[['방문점포종류수평균']] = \
(cust_15_half1_2[['방문점포종류수']] + cust_14_half2_2[['방문점포종류수']])/2
x_test[['방문점포종류수변동']] = \
cust_15_half1_2[['방문점포종류수']] - cust_14_half2_2[['방문점포종류수']]
x_test[['회당구매금액평균']] = \
(cust_15_half1_2[['회당구매금액']] + cust_14_half2_2[['회당구매금액']])/2
x_test[['회당구매금액변동']] = \
cust_15_half1_2[['회당구매금액']] - cust_14_half2_2[['회당구매금액']]
x_test['recency'] = \
(pd.to_datetime('20150701', format = '%Y%m%d') - cust_15_half1_2['마지막구매일자']).apply(lambda x: x.days)
x_test = x_test.drop(['A구매금액비율', 'B구매금액비율', 'C구매금액비율', 'D구매금액비율', 'A방문횟수',
       'B방문횟수', 'C방문횟수', 'D방문횟수', 'A상품구매개수', 'B상품구매개수', 'C상품구매개수', 'D상품구매개수',
       '총구매금액', '방문점포종류수', '회당구매금액', '마지막구매일자'], axis = 1)
x_test

,성별,멤버십가입여부,구매상품종류수,연령대_19세이하,연령대_20세~24세,연령대_25세~29세,연령대_30세~34세,연령대_35세~39세,연령대_40세~44세,연령대_45세~49세,연령대_50세~54세,연령대_55세~59세,연령대_60세이상,주요구매시간대_낮,주요구매시간대_야간,주요구매시간대_오전,주요구매시간대_오후,주요구매시간대_저녁,A구매금액비율평균,B구매금액비율평균,C구매금액비율평균,D구매금액비율평균,A구매금액비율변동,B구매금액비율변동,C구매금액비율변동,D구매금액비율변동,총구매금액평균,총구매금액변동,A방문횟수평균,B방문횟수평균,C방문횟수평균,D방문횟수평균,A방문횟수변동,B방문횟수변동,C방문횟수변동,D방문횟수변동,A상품구매개수평균,B상품구매개수평균,C상품구매개수평균,D상품구매개수평균,A상품구매개수변동,B상품구매개수변동,C상품구매개수변동,D상품구매개수변동,방문점포종류수평균,방문점포종류수변동,회당구매금액평균,회당구매금액변동,recency
고객번호,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0,134.0,0,0,0,0,0,0,0,0,0,1,2.0,0.0,0.0,0.0,0.0,0.974223,0.011327,0.014450,0.000000,-0.006062,0.000604,0.005458,0.000000,2.411721e+07,-1.477392e+07,189.5,4.0,8.5,0.0,-33.0,0.0,-1.0,0.0,258.5,26.5,49.0,0.0,-31.0,-29.0,16.0,0.0,6.5,-1.0,117144.236846,-53420.871532,4
2,1,0,80.0,0,0,0,0,0,0,0,0,0,1,2.0,0.0,0.0,0.0,0.0,0.993454,0.003443,0.000000,0.003103,-0.008854,0.002649,0.000000,0.006205,2.311072e+07,5.734210e+06,161.5,2.0,0.0,3.0,-9.0,2.0,0.0,6.0,251.0,7.5,0.0,7.5,-26.0,1.0,0.0,15.0,4.0,2.0,138856.113138,35273.669855,7
3,1,0,54.0,0,0,0,0,0,0,0,0,0,1,2.0,0.0,0.0,0.0,0.0,0.000000,0.003338,0.996662,0.000000,0.000000,-0.006676,0.006676,0.000000,7.581779e+05,-4.091045e+05,0.0,0.5,130.5,0.0,0.0,-1.0,-73.0,0.0,0.0,1.0,304.5,0.0,0.0,-2.0,-169.0,0.0,2.0,-2.0,5810.085688,159.098258,2
4,0,0,108.0,0,0,0,0,0,0,0,0,0,1,2.0,0.0,0.0,0.0,0.0,0.869409,0.095060,0.035532,0.000000,-0.004699,-0.007527,0.012226,0.000000,3.674901e+06,-5.132587e+05,98.0,10.0,11.5,0.0,28.0,-4.0,5.0,0.0,146.0,39.0,32.5,0.0,82.0,-6.0,15.0,0.0,4.5,1.0,31476.322132,-11933.657505,1
5,1,0,25.0,0,0,0,0,0,0,0,0,0,1,2.0,0.0,0.0,0.0,0.0,0.997010,0.000000,0.000000,0.002990,-0.005980,0.000000,0.000000,0.005980,2.399351e+06,6.880704e+05,76.5,0.0,0.0,1.5,3.0,0.0,0.0,3.0,124.5,0.0,0.0,2.0,-11.0,0.0,0.0,4.0,1.5,1.0,30636.583909,6464.755432,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19378,0,0,79.0,0,0,0,0,0,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
19379,0,1,92.0,0,1,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,1.0,0.343378,0.042424,0.011388,0.602809,-0.102580,0.084848,0.022776,-0.005045,1.669506e+06,1.140094e+06,9.0,9.0,1.5,87.5,6.0,18.0,3.0,75.0,9.0,9.5,11.0,145.0,6.0,19.0,22.0,128.0,5.5,5.0,16903.796518,-5458.812951,1
19380,0,1,36.0,0,1,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
# 분기별 카테고리 랭크 데이터 분리
q1 = cust_14_half2_2.iloc[:,19:31]
q2 = cust_14_half2_2.iloc[:,31:43]
q3 = cust_15_half1_2.iloc[:,19:31]
q4 = cust_15_half1_2.iloc[:,31:43]

In [ ]:
# 분기별 카테고리 랭크 컬럼명 통일
q1 = q1.rename({'1_1':'1', '2_1':'2', '3_1':'3', '4_1':'4',
       '5_1':'5', '6_1':'6', '7_1':'7', '8_1':'8', '9_1':'9', '10_1':'10', '11_1':'11', '12_1':'12'}, axis =1)
q2 = q2.rename({'1_2':'1', '2_2':'2', '3_2':'3', '4_2':'4',
       '5_2':'5', '6_2':'6', '7_2':'7', '8_2':'8', '9_2':'9', '10_2':'10', '11_2':'11', '12_2':'12'}, axis =1)
q3 = q3.rename({'1_1':'1', '2_1':'2', '3_1':'3', '4_1':'4',
       '5_1':'5', '6_1':'6', '7_1':'7', '8_1':'8', '9_1':'9', '10_1':'10', '11_1':'11', '12_1':'12'}, axis =1)
q4 = q4.rename({'1_2':'1', '2_2':'2', '3_2':'3', '4_2':'4',
       '5_2':'5', '6_2':'6', '7_2':'7', '8_2':'8', '9_2':'9', '10_2':'10', '11_2':'11', '12_2':'12'}, axis =1)

In [ ]:
# 카테고리 net 반영
x_test = x_test.merge((q4-q1).rename({'1':'1_net', '2':'2_net', '3':'3_net', '4':'4_net', '5':'5_net', '6':'6_net', '7':'7_net', '8':'8_net', '9':'9_net', '10':'10_net', '11':'11_net', '12':'12_net'}, axis = 1), how = 'left', on = '고객번호')

In [ ]:
# 카테고리 sum 반영
x_test = x_test.merge(((q4 - q3).apply(abs) + (q3 - q2).apply(abs) + (q2 - q1).apply(abs)).rename({'1':'1_sum', '2':'2_sum', '3':'3_sum', '4':'4_sum', '5':'5_sum', '6':'6_sum', '7':'7_sum', '8':'8_sum', '9':'9_sum', '10':'10_sum', '11':'11_sum', '12':'12_sum'}, axis = 1), how = 'left', on = '고객번호')

In [ ]:
# 독립변수 null 값 있는 행 제거
x_train.dropna(inplace = True)
x_test.dropna(inplace = True)

# Scaling

In [ ]:
# 스케일링
# 훈련용 데이터셋 Fit
# 훈련용, 테스트용 transform
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

ss.fit(x_train[['A구매금액비율평균', 'B구매금액비율평균', 'C구매금액비율평균', 'D구매금액비율평균', 'A구매금액비율변동',
       'B구매금액비율변동', 'C구매금액비율변동', 'D구매금액비율변동', '총구매금액평균', '총구매금액변동', 'A방문횟수평균',
       'B방문횟수평균', 'C방문횟수평균', 'D방문횟수평균', 'A방문횟수변동', 'B방문횟수변동', 'C방문횟수변동',
       'D방문횟수변동', 'A상품구매개수평균', 'B상품구매개수평균', 'C상품구매개수평균', 'D상품구매개수평균',
       'A상품구매개수변동', 'B상품구매개수변동', 'C상품구매개수변동', 'D상품구매개수변동', '방문점포종류수평균',
       '방문점포종류수변동', '회당구매금액평균',
       '회당구매금액변동', 'recency', '1_net', '2_net', '3_net', '4_net', '5_net',
       '6_net', '7_net', '8_net', '9_net', '10_net', '11_net', '12_net',
       '1_sum', '2_sum', '3_sum', '4_sum', '5_sum', '6_sum', '7_sum', '8_sum',
       '9_sum', '10_sum', '11_sum', '12_sum']])

x_train[['A구매금액비율평균', 'B구매금액비율평균', 'C구매금액비율평균', 'D구매금액비율평균', 'A구매금액비율변동',
       'B구매금액비율변동', 'C구매금액비율변동', 'D구매금액비율변동', '총구매금액평균', '총구매금액변동', 'A방문횟수평균',
       'B방문횟수평균', 'C방문횟수평균', 'D방문횟수평균', 'A방문횟수변동', 'B방문횟수변동', 'C방문횟수변동',
       'D방문횟수변동', 'A상품구매개수평균', 'B상품구매개수평균', 'C상품구매개수평균', 'D상품구매개수평균',
       'A상품구매개수변동', 'B상품구매개수변동', 'C상품구매개수변동', 'D상품구매개수변동', '방문점포종류수평균',
       '방문점포종류수변동', '회당구매금액평균',
       '회당구매금액변동', 'recency', '1_net', '2_net', '3_net', '4_net', '5_net',
       '6_net', '7_net', '8_net', '9_net', '10_net', '11_net', '12_net',
       '1_sum', '2_sum', '3_sum', '4_sum', '5_sum', '6_sum', '7_sum', '8_sum',
       '9_sum', '10_sum', '11_sum', '12_sum']] = \
ss.transform(x_train[['A구매금액비율평균', 'B구매금액비율평균', 'C구매금액비율평균', 'D구매금액비율평균', 'A구매금액비율변동',
       'B구매금액비율변동', 'C구매금액비율변동', 'D구매금액비율변동', '총구매금액평균', '총구매금액변동', 'A방문횟수평균',
       'B방문횟수평균', 'C방문횟수평균', 'D방문횟수평균', 'A방문횟수변동', 'B방문횟수변동', 'C방문횟수변동',
       'D방문횟수변동', 'A상품구매개수평균', 'B상품구매개수평균', 'C상품구매개수평균', 'D상품구매개수평균',
       'A상품구매개수변동', 'B상품구매개수변동', 'C상품구매개수변동', 'D상품구매개수변동', '방문점포종류수평균',
       '방문점포종류수변동', '회당구매금액평균',
       '회당구매금액변동', 'recency', '1_net', '2_net', '3_net', '4_net', '5_net',
       '6_net', '7_net', '8_net', '9_net', '10_net', '11_net', '12_net',
       '1_sum', '2_sum', '3_sum', '4_sum', '5_sum', '6_sum', '7_sum', '8_sum',
       '9_sum', '10_sum', '11_sum', '12_sum']])


x_test[['A구매금액비율평균', 'B구매금액비율평균', 'C구매금액비율평균', 'D구매금액비율평균', 'A구매금액비율변동',
       'B구매금액비율변동', 'C구매금액비율변동', 'D구매금액비율변동', '총구매금액평균', '총구매금액변동', 'A방문횟수평균',
       'B방문횟수평균', 'C방문횟수평균', 'D방문횟수평균', 'A방문횟수변동', 'B방문횟수변동', 'C방문횟수변동',
       'D방문횟수변동', 'A상품구매개수평균', 'B상품구매개수평균', 'C상품구매개수평균', 'D상품구매개수평균',
       'A상품구매개수변동', 'B상품구매개수변동', 'C상품구매개수변동', 'D상품구매개수변동', '방문점포종류수평균',
       '방문점포종류수변동', '회당구매금액평균',
       '회당구매금액변동', 'recency', '1_net', '2_net', '3_net', '4_net', '5_net',
       '6_net', '7_net', '8_net', '9_net', '10_net', '11_net', '12_net',
       '1_sum', '2_sum', '3_sum', '4_sum', '5_sum', '6_sum', '7_sum', '8_sum',
       '9_sum', '10_sum', '11_sum', '12_sum']] = \
ss.transform(x_test[['A구매금액비율평균', 'B구매금액비율평균', 'C구매금액비율평균', 'D구매금액비율평균', 'A구매금액비율변동',
       'B구매금액비율변동', 'C구매금액비율변동', 'D구매금액비율변동', '총구매금액평균', '총구매금액변동', 'A방문횟수평균',
       'B방문횟수평균', 'C방문횟수평균', 'D방문횟수평균', 'A방문횟수변동', 'B방문횟수변동', 'C방문횟수변동',
       'D방문횟수변동', 'A상품구매개수평균', 'B상품구매개수평균', 'C상품구매개수평균', 'D상품구매개수평균',
       'A상품구매개수변동', 'B상품구매개수변동', 'C상품구매개수변동', 'D상품구매개수변동', '방문점포종류수평균',
       '방문점포종류수변동', '회당구매금액평균',
       '회당구매금액변동', 'recency', '1_net', '2_net', '3_net', '4_net', '5_net',
       '6_net', '7_net', '8_net', '9_net', '10_net', '11_net', '12_net',
       '1_sum', '2_sum', '3_sum', '4_sum', '5_sum', '6_sum', '7_sum', '8_sum',
       '9_sum', '10_sum', '11_sum', '12_sum']])

In [ ]:
# 종속변수 길이와 독립변수 길이 맞춰주기
y_train = x_train.merge(y_train, on = '고객번호', how = 'left')['구매증감']
y_test = x_test.merge(y_test, on = '고객번호', how = 'left')['구매증감']

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(18931, 73)
(18931,)
(19135, 73)
(19135,)


## csv 파일 업데이트: x_train, x_test, y_train, y_test

In [ ]:
x_train.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/x_train.csv')
x_test.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/x_test.csv')
y_train.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/y_train.csv')
y_test.to_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/y_test.csv')

# 모델링, 스코어 파악
- Boosting(XGB, lgbm, Cat, AdaBoost), Logistic Regression, Decision Tree, RF

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

x_train = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/x_train.csv', index_col=0)
x_test = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/x_test.csv', index_col=0)
y_train = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/y_train.csv', index_col=0)
y_test = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/y_test.csv', index_col=0)

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.4 MB 1.2 MB/s 


In [ ]:
# XGBoost
from xgboost import XGBClassifier
model = XGBClassifier()
model.fit(x_train, y_train)
print('XGB train 정확도: {0:.4f}'.format(model.score(x_train, y_train)))
print('XGB test 정확도: {0:.4f}'.format(model.score(x_test, y_test)))
print()

# lightgbm
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(x_train, y_train)
print('LightGBM train 정확도: {0:.4f}'.format(model.score(x_train, y_train)))
print('LightGBM test 정확도: {0:.4f}'.format(model.score(x_test, y_test)))
print()

# catboost
from catboost import CatBoostClassifier
model = CatBoostClassifier(silent=True)
model.fit(x_train, y_train)
print('CatBoost train 정확도: {0:.4f}'.format(model.score(x_train, y_train)))
print('CatBoost test 정확도: {0:.4f}'.format(model.score(x_test, y_test)))
print()

# AdaBoost
from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(x_train, y_train)
print('AdaBoost train 정확도: {:.4f}'.format(model.score(x_train, y_train)))
print('AdaBoost test 정확도: {:.4f}'.format(model.score(x_test, y_test)))
print()

# LogisticRegression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)
print('LogisticRegression train 정확도: {0:.4f}'.format(model.score(x_train, y_train)))
print('LogisticRegression test 정확도: {0:.4f}'.format(model.score(x_test, y_test)))
print()

# DecisionTree
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
print("DecisionTree train 정확도: {0:.4f}".format(model.score(x_train, y_train)))
print("DecisionTree test 정확도: {0:.4f}".format(model.score(x_test, y_test)))
print()

# RandomForest
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)
print('RandomForest train 정확도: {0:.4f}'.format(model.score(x_train, y_train)))
print('RandomForest train 정확도: {0:.4f}'.format(model.score(x_test, y_test)))

XGB train 정확도: 0.6762
XGB test 정확도: 0.6240

LightGBM train 정확도: 0.7608
LightGBM test 정확도: 0.6191

CatBoost train 정확도: 0.7844
CatBoost test 정확도: 0.6213

AdaBoost train 정확도: 0.6577
AdaBoost test 정확도: 0.6194

LogisticRegression train 정확도: 0.6447
LogisticRegression test 정확도: 0.5098

DecisionTree train 정확도: 1.0000
DecisionTree test 정확도: 0.5562



In [ ]:
# Decision Tree 사용한 코드
# mc_j : 다중공선성 가진 컬럼들 drop한 데이터
# mc_s : 다중공선성 가진 컬럼들 drop 안한 데이터

# 불러오기. 
import pandas as pd
x_test_mc_j = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/ProcessedData/x_test_j_scaled.csv', encoding='utf-8', index_col=0)
x_test_mc_s= pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/ProcessedData/x_test_s_scaled.csv', encoding='utf-8', index_col=0)
x_train_mc_j= pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/ProcessedData/x_train_j_scaled.csv', encoding='utf-8', index_col=0)
x_train_mc_s= pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/ProcessedData/x_train_s_scaled.csv', encoding='utf-8', index_col=0)

y_train = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/y_train.csv', encoding='utf-8', index_col=0)
y_test = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/y_test.csv', encoding='utf-8', index_col=0)

# 학습 수행
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(criterion='gini', max_depth=3,random_state=156)
dt_clf.fit(x_train_mc_j,y_train)

# 예측 수행
pred = dt_clf.predict(x_test_mc_s)

In [ ]:
# Decision Tree 사용한 코드
from sklearn.metrics import accuracy_score
print("예측 정확도: {0:.4f}".format(accuracy_score(y_test,pred)))

In [ ]:
# Decision Tree 사용한 코드 (시각화)
import matplotlib.pyplot as plt
from sklearn import tree

plt.rc('font', family='NanumBarunGothic') 
plt.figure(figsize = (20,15))
tree.plot_tree(dt_clf,filled=True,
              feature_names=x_train_mc_j.columns,
              rounded=True,fontsize=14)
              
plt.show()

# 변수 선택

In [ ]:
#피처중요도 파악하기 

import pandas as pd
import numpy as np
import seaborn as sns
from patsy import dmatrices
import statsmodels.api as sm;
from statsmodels.stats.outliers_influence import variance_inflation_factor
%matplotlib inline


#파일 불러오기
x_train = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/x_train.csv', index_col=0)
x_test = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/x_test.csv', index_col=0)
y_train = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/y_train.csv', index_col=0)
y_test = pd.read_csv('/content/drive/MyDrive/1차 프로젝트/RAW DATA/y_test.csv', index_col=0)

In [ ]:
#피처중요도 파악 모듈 불러오기 
!pip install eli5

     |████████████████████████████████| 106 kB 12.5 MB/s 


In [ ]:
# XGBoost 결과 (train)
from xgboost import XGBClassifier
import eli5
from eli5.sklearn import PermutationImportance

xgb = XGBClassifier()
xgb.fit(x_train, y_train)
print('XGB train 정확도(B): {0:.4f}'.format(xgb.score(x_train, y_train)))

permuter = PermutationImportance(xgb, random_state=0).fit(x_train, y_train)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

#신규변수가 중요도가 높게 들어감 : 총구매금액변동, 회당구매금액변동, ,총구매금액평균,A방문횟수변동

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGB train 정확도(B): 0.6788


Weight,Feature
0.0514 ± 0.0049,총구매금액변동
0.0288 ± 0.0023,총구매금액평균
0.0272 ± 0.0012,A방문횟수평균
0.0253 ± 0.0016,B방문횟수평균
0.0159 ± 0.0012,C방문횟수평균
0.0073 ± 0.0025,회당구매금액변동
0.0066 ± 0.0016,A구매금액비율변동
0.0055 ± 0.0015,A상품구매개수평균
0.0033 ± 0.0017,C상품구매개수평균
0.0023 ± 0.0014,A방문횟수변동


In [ ]:
# XGBoost 결과 (test)
print('XGB test 정확도(B): {0:.4f}'.format(xgb.score(x_test, y_test)))
permuter = PermutationImportance(xgb, random_state=0).fit(x_test, y_test)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

XGB test 정확도(B): 0.6190


Weight,Feature
0.0397 ± 0.0041,총구매금액변동
0.0285 ± 0.0029,B방문횟수평균
0.0150 ± 0.0013,C방문횟수평균
0.0089 ± 0.0019,회당구매금액변동
0.0085 ± 0.0037,A방문횟수평균
0.0055 ± 0.0035,총구매금액평균
0.0047 ± 0.0031,A구매금액비율변동
0.0022 ± 0.0013,A상품구매개수평균
0.0012 ± 0.0012,C상품구매개수평균
0.0010 ± 0.0003,D방문횟수평균


In [ ]:
# lightgbm 결과 (train)
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(x_train, y_train)
print('LightGBM train 정확도: {0:.4f}'.format(model.score(x_train, y_train)))

permuter = PermutationImportance(model, random_state=0).fit(x_train, y_train)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LightGBM train 정확도: 0.7622


Weight,Feature
0.0640 ± 0.0054,총구매금액변동
0.0526 ± 0.0041,B방문횟수평균
0.0505 ± 0.0044,총구매금액평균
0.0481 ± 0.0027,A방문횟수평균
0.0304 ± 0.0014,C방문횟수평균
0.0236 ± 0.0018,회당구매금액변동
0.0200 ± 0.0019,A상품구매개수평균
0.0171 ± 0.0023,A구매금액비율변동
0.0149 ± 0.0018,회당구매금액평균
0.0131 ± 0.0026,A방문횟수변동


In [ ]:
# lightgbm 결과 (test)
print('LightGBM train 정확도: {0:.4f}'.format(model.score(x_test, y_test)))
permuter = PermutationImportance(xgb, random_state=0).fit(x_test, y_test)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

LightGBM train 정확도: 0.6217


Weight,Feature
0.0397 ± 0.0041,총구매금액변동
0.0285 ± 0.0029,B방문횟수평균
0.0150 ± 0.0013,C방문횟수평균
0.0089 ± 0.0019,회당구매금액변동
0.0085 ± 0.0037,A방문횟수평균
0.0055 ± 0.0035,총구매금액평균
0.0047 ± 0.0031,A구매금액비율변동
0.0022 ± 0.0013,A상품구매개수평균
0.0012 ± 0.0012,C상품구매개수평균
0.0010 ± 0.0003,D방문횟수평균


In [ ]:
# catboost 사용하기 위한 설치 
!pip install catboost

     |████████████████████████████████| 76.4 MB 49 kB/s 


In [ ]:
#catboost 결과 (train) 
from catboost import CatBoostClassifier

cb = CatBoostClassifier(silent=True)
cb.fit(x_train, y_train)
cb.score(x_test, y_test)
print('CatBoost train 정확도(B): {0:.4f}'.format(cb.score(x_train, y_train)))

permuter = PermutationImportance(cb, random_state=0).fit(x_train, y_train)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

CatBoost train 정확도(B): 0.7828


Weight,Feature
0.0665 ± 0.0034,B방문횟수평균
0.0519 ± 0.0052,총구매금액변동
0.0451 ± 0.0021,A방문횟수평균
0.0333 ± 0.0017,총구매금액평균
0.0324 ± 0.0027,C방문횟수평균
0.0208 ± 0.0028,회당구매금액변동
0.0178 ± 0.0016,A상품구매개수평균
0.0165 ± 0.0037,회당구매금액평균
0.0140 ± 0.0031,B방문횟수변동
0.0129 ± 0.0006,C상품구매개수평균


In [ ]:
#catboost 결과 (test)
cb = CatBoostClassifier(silent=True)
cb.fit(x_train, y_train)
cb.score(x_test, y_test)
print('CatBoost test 정확도(B): {0:.4f}'.format(cb.score(x_test, y_test)))

permuter = PermutationImportance(cb, random_state=0).fit(x_test, y_test)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

CatBoost test 정확도(B): 0.6252


Weight,Feature
0.0430 ± 0.0014,B방문횟수평균
0.0313 ± 0.0059,총구매금액변동
0.0196 ± 0.0024,A방문횟수평균
0.0179 ± 0.0033,C방문횟수평균
0.0152 ± 0.0023,회당구매금액변동
0.0094 ± 0.0014,A상품구매개수평균
0.0048 ± 0.0034,총구매금액평균
0.0044 ± 0.0015,C상품구매개수평균
0.0040 ± 0.0012,A구매금액비율변동
0.0024 ± 0.0011,방문점포종류수평균


In [ ]:
#AdaBoost 결과 (train)
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

model = AdaBoostClassifier(n_estimators=30, 
                        random_state=10, 
                        learning_rate=0.1)
model.fit(x_train, y_train)
print('AdaBoost train 정확도: {:.4f}'.format(model.score(x_train, y_train)))

permuter = PermutationImportance(model, random_state=0).fit(x_train, y_train)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoost train 정확도: 0.6283


Weight,Feature
0.0705 ± 0.0075,총구매금액변동
0.0066 ± 0.0017,회당구매금액변동
0.0056 ± 0.0023,총구매금액평균
0.0040 ± 0.0014,A방문횟수평균
0.0022 ± 0.0006,B방문횟수평균
0.0010 ± 0.0014,A구매금액비율변동
0.0010 ± 0.0005,C방문횟수평균
0 ± 0.0000,B방문횟수변동
0 ± 0.0000,A방문횟수변동
0 ± 0.0000,D방문횟수평균


In [ ]:
#AdaBoost 결과 (test)
print('AdaBoost test 정확도: {:.4f}'.format(model.score(x_test, y_test)))

permuter = PermutationImportance(model, random_state=0).fit(x_test, y_test)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

AdaBoost test 정확도: 0.5897


Weight,Feature
0.0609 ± 0.0064,총구매금액변동
0.0106 ± 0.0035,회당구매금액변동
0.0012 ± 0.0009,B방문횟수평균
0.0010 ± 0.0012,C방문횟수평균
0.0008 ± 0.0009,A구매금액비율변동
0 ± 0.0000,연령대_40세~44세
0 ± 0.0000,C방문횟수변동
0 ± 0.0000,B방문횟수변동
0 ± 0.0000,A방문횟수변동
0 ± 0.0000,D방문횟수평균


In [ ]:
#LogisticRegression 결과 (train)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(x_train, y_train)
print('LogisticRegression train 정확도: {0:.4f}'.format(model.score(x_train, y_train)))

permuter = PermutationImportance(model, random_state=0).fit(x_train, y_train)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression train 정확도: 0.6386


Weight,Feature
0.0573 ± 0.0032,B방문횟수평균
0.0386 ± 0.0033,A방문횟수평균
0.0380 ± 0.0019,C방문횟수평균
0.0180 ± 0.0035,A상품구매개수평균
0.0103 ± 0.0029,총구매금액평균
0.0071 ± 0.0039,B구매금액비율평균
0.0062 ± 0.0019,총구매금액변동
0.0055 ± 0.0015,A구매금액비율변동
0.0054 ± 0.0015,A구매금액비율평균
0.0052 ± 0.0018,A방문횟수변동


In [ ]:
#LogisticRegression 결과 (test)
print('LogisticRegression test 정확도: {0:.4f}'.format(model.score(x_test, y_test)))

permuter = PermutationImportance(model, random_state=0).fit(x_test, y_test)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

LogisticRegression test 정확도: 0.5850


Weight,Feature
0.0460 ± 0.0036,B방문횟수평균
0.0280 ± 0.0033,C방문횟수평균
0.0105 ± 0.0042,A방문횟수평균
0.0083 ± 0.0017,총구매금액변동
0.0065 ± 0.0016,A구매금액비율변동
0.0045 ± 0.0011,회당구매금액변동
0.0043 ± 0.0043,A방문횟수변동
0.0042 ± 0.0031,B상품구매개수변동
0.0034 ± 0.0024,C상품구매개수평균
0.0029 ± 0.0021,C구매금액비율변동


In [ ]:
#DecisionTree 결과 (train) 

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

tree = DecisionTreeClassifier(random_state=0)
tree.fit(x_train, y_train)
print("디시전트리 train 정확도: {0:.4f}".format(tree.score(x_train, y_train)))

permuter = PermutationImportance(tree, random_state=0).fit(x_train, y_train)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())

디시전트리 train 정확도: 1.0000


Weight,Feature
0.3496 ± 0.0026,총구매금액변동
0.1851 ± 0.0041,A방문횟수평균
0.1757 ± 0.0015,총구매금액평균
0.1639 ± 0.0038,B방문횟수평균
0.1597 ± 0.0030,회당구매금액변동
0.1124 ± 0.0020,A상품구매개수평균
0.0930 ± 0.0018,A구매금액비율변동
0.0900 ± 0.0020,회당구매금액평균
0.0855 ± 0.0010,C방문횟수평균
0.0660 ± 0.0017,C구매금액비율평균


In [ ]:
#DecisionTree 결과 (test) 
print("디시전트리 test 정확도: {0:.4f}".format(tree.score(x_test, y_test)))
permuter = PermutationImportance(tree, random_state=0).fit(x_test, y_test)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())


디시전트리 test 정확도: 0.5470


Weight,Feature
0.0233 ± 0.0061,총구매금액변동
0.0118 ± 0.0014,B방문횟수평균
0.0099 ± 0.0011,회당구매금액평균
0.0088 ± 0.0031,A방문횟수평균
0.0085 ± 0.0016,A상품구매개수평균
0.0049 ± 0.0024,C방문횟수평균
0.0034 ± 0.0031,A방문횟수변동
0.0031 ± 0.0017,C구매금액비율변동
0.0029 ± 0.0027,회당구매금액변동
0.0023 ± 0.0011,B방문횟수변동


In [ ]:
#랜덤포레스트 결과(train) 

from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)
print('RandomForestClassifier train 정확도: {0:.4f}'.format(model.score(x_train, y_train)))

permuter = PermutationImportance(model, random_state=0).fit(x_train, y_train)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier train 정확도: 1.0000


Weight,Feature
0.0473 ± 0.0030,총구매금액변동
0.0295 ± 0.0007,A방문횟수평균
0.0255 ± 0.0017,B방문횟수평균
0.0229 ± 0.0011,총구매금액평균
0.0228 ± 0.0017,회당구매금액변동
0.0113 ± 0.0006,A상품구매개수평균
0.0109 ± 0.0004,C방문횟수평균
0.0042 ± 0.0007,A구매금액비율변동
0.0016 ± 0.0004,C상품구매개수평균
0.0016 ± 0.0002,A방문횟수변동


In [ ]:
#랜덤포레스트 결과(test) 
print('RandomForestClassifier test 정확도: {0:.4f}'.format(model.score(x_test, y_test)))

permuter = PermutationImportance(model, random_sta4te=0).fit(x_test, y_test)
eli5.show_weights(permuter, feature_names=x_train.columns.tolist())


RandomForestClassifier test 정확도: 0.6096


Weight,Feature
0.0218 ± 0.0062,총구매금액변동
0.0195 ± 0.0024,B방문횟수평균
0.0117 ± 0.0015,회당구매금액변동
0.0090 ± 0.0019,A방문횟수평균
0.0086 ± 0.0020,C방문횟수평균
0.0026 ± 0.0019,B상품구매개수평균
0.0022 ± 0.0017,9_net
0.0021 ± 0.0039,총구매금액평균
0.0018 ± 0.0018,A구매금액비율변동
0.0018 ± 0.0029,A상품구매개수평균


# 군집화